# Comparing Strings

There are many ways in which one may compare two or more strings. Typical goals of such comparisions are to indicate how similar (or dissimilar) two strings are. This is typically done by defining a **metric** (or **distance**) between two words. 

In this notebook we consider several ways to compare strings. These are well-known distances such as the **Hamming** distance, **longest common subsequence**, among others. 

## Longest Common Subsequence (LCS)

The **Longest Common Subsequence (LCS)** problem askes to determine the longest subsequence of two strings A and B.
That is, over all subsequence of characters for A and B, what is the longest such subsequence shared between the two strings. 
Note that characters need not be consequitve (i.e. need not be a substring of characters) only that one appears before the other in both strings. 

We can define the length of the LCS of A and B recursively. Suppose A and B have lengths a and b respectively. Then either the last characters of both strings are equal, or they are note. 

If $A[a] == B[b]$ then this character will belong to the LCS. We then "prune" these characters and ask what is the LCS of $A[1:a-1]$ and $B[1:b-1]$.

Otherwise this character will not belong the LCS. Thus we can consider prunning both characters (one at a time) and consider the LCS of thw two subproblems. As we want the longest common subsequence, we maximize over the lengths of the resulting LCS for the subproblems. 

The base case for this problem is when either string is empty; trivially the length of the LCS is 0. For $a,b > 0$ our recurrence relationship is:

$$
\begin{align}
LCS(A[1:i],B[1:j]) &=
\begin{cases}
0 &\text{if $i==0$ or $j==0$} \\
1 + LCS(A[1:i-1],B[1:j-1]) &\text{if $A[i]==B[j], i,j>0$} \\
\max \{ LCS(A[1:i], B[1:j-1]), LCS(A[1:i-1], B[1:j])\} &\text{otherwise}
\end{cases}
\end{align}
$$

In [2]:
class LCS_Entry:
#     __slots__ = ('value', 'lcs', 'is_null',)
    def __init__ (self):
        self.value = -1
        self.lcs = ""
        
    def __str__(self):
        return str(self.value) + ':' + self.lcs
    
    def __repr__(self):
        return self.__str__()
    

In [3]:
# Below is a top-down (memoized) and bottom-up (DP)  implementation of the LCS recurrence relationship.

def LCS_TD(i,j, A, B):

    # Need to set value.
    if dp[i][j].value == -1:
        
        # Base cases
        if i == 0 or j == 0:
            dp[i][j].value = 0
            dp[i][j].lcs = ""
            return dp[i][j]
        
        # Non base cases
        # Characters match, add to LCS
        if A[i-1] == B[j-1]:
            result = LCS_TD(i-1, j-1, A, B)
            dp[i][j].value = 1 + result.value
            dp[i][j].lcs = str(result.lcs) + A[i-1]
            
        # Prune each ending character, see which results in lcs.
        else:
            result_1 = LCS_TD(i-1, j, A, B)
            result_2 = LCS_TD(i, j-1, A, B)
            if result_1.value >= result_2.value:
                result = result_1
            else:
                result = result_2
            dp[i][j].value = result.value
            dp[i][j].lcs = str(result.lcs)
            
    return dp[i][j]


def LCS_BU(A, B):
    
    # Base cases
    for i in range(len(A)+1):
        dp[i][0].value = 0
        dp[i][0].lcs = ""
    
    for j in range(len(B)+1):
        dp[0][j].value = 0
        dp[0][j].lcs = ""
    
    # Non base cases
    for i in range(1,len(A)+1):
        for j in range(1,len(B)+1):
            if A[i-1] == B[j-1]:
                dp[i][j].value = 1 + dp[i-1][j-1].value
                dp[i][j].lcs =  str(dp[i-1][j-1].lcs) + A[i-1]
            
            else:
                result_1 = dp[i-1][j]
                result_2 = dp[i][j-1]
                result = None
                if result_1.value >= result_2.value:
                    result = result_1
                else:
                    result = result_2
                
                dp[i][j].value = result.value
                dp[i][j].lcs = str(result.lcs)
    
    return dp[len(A)][len(B)]

In [4]:

# Examples LCS
A = "subsequence"
B = "supercan"

a = len(A)
b = len(B)

# Create dp matrix 
dp = [[LCS_Entry() for j in range(b+1)] for i in range(a+1)]

result = LCS_TD(a, b, A, B)
print(result)

result = LCS_BU(A, B)
print(result)

4:suen
4:suen


## Minimum LCS Distance

The **minimum LCS (min LCS) distance** between two strings A and B is the minimum number of insertion and deletion operations to conver string A to string B. The naming of such a distance comes from its relationship with the LCS of A and B. 

Indeed if A and B have length a and b respectively, then an upper bound on the min LCS distance is $a+b$; remove all a characters from A then insert b characters of B. However any subsequence of characters that is common to each string need not be removed. Thus when counting the number of removal and insertion operations, for each character that is common between the two need not be removed. Thus we have

$$ 
min LCS distance = a + b - 2LCS(A,B)
$$

# Levenshtein Distance

 The **Levenshtein distance (LEV)** of two strings A and B is the minimum number of operations to convert A to B, where the operations allowed are the insertion, deletion, or substitution of a single character.
 
If the lengths of the string are $a$ and $b$, we can observe the following about the lev distance of A and B.  
1) lev(A,B) = 0 if and only if the A=B.  
2) lev(A,B) is at most $\max\{ a,b \}$, as we can preform the substitution, then addition to match the longer string.  
3) lev(A,B) is at least $|a-b|$, as we would need at least this many insertion operations to fill the "gap" between the two. 

We can define the Levelshtein distance using a recursive relationship where we consider the three possible operations, their results, and minimize over the two. For our bases cases (if one string is empty) we require the length of the other string operations to match the two. Thus we have


$$
\begin{align}
LEV(A[1:i],B[1:j]) &=
\begin{cases}
\max\{ i,j \} &\text{if $i==0$ or $j==0$} \\
\min
\begin{cases}
LEV(A[1:i], B[1:j-1]) + 1, \\
LEV(A[1:i-1], B[1:j]) + 1, \\
LEV(A[1:i-1], B[1:j-1]) + \unicode{x1D7D9}_{A[i]\neq B[j]}
\end{cases}
&\text{otherwise}
\end{cases}
\end{align}
$$

where (by converting A to B) the entries of the min operator correspond to insertion, deletion, and either a substituion (if $A[i] \neq B[j]$) or no operation being taken.

In [74]:
class LEV_Entry:
    
    def __init__ (self):
        self.value = -1
        self.operations = list()
        
    def __str__(self):
        return str(self.value) + ':' + str(self.operations)
    
    def __repr__(self):
        return self.__str__()

In [103]:
def LEV_BU(A, B):
    
    # Base cases
    for i in range(1, len(A)+1):
        dp[i][0].value = i
        dp[i][0].operations.append(''.join(dp[i-1][0].operations) + ' del ' + A[i-1] + " ")
    
    for j in range(1, len(B)+1):
        dp[0][j].value = j
        dp[0][j].operations.append(''.join(dp[0][j-1].operations) + ' ins ' + B[j-1] + " ")
        
    # Non base cases
    for i in range(1,len(A)+1):
        for j in range(1,len(B)+1):
            
            # Keep charcter, no action.
            if A[i-1] == B[j-1]:
                result_sub = dp[i-1][j-1].value
                action = " keep " + A[i-1]
            else:
                result_sub = 1 + dp[i-1][j-1].value
                action = " sub " + A[i-1] + " for " + B[j-1]
            
            result_insert = 1 + dp[i][j-1].value
            result_delete = 1 + dp[i-1][j].value
            
            if result_sub <= result_insert and result_sub <= result_delete:
                dp[i][j].value = result_sub
                dp[i][j].operations.append(''.join(dp[i-1][j-1].operations) + action +" ")
            elif result_insert <= result_sub and result_insert <= result_delete:
                dp[i][j].value = result_insert
                dp[i][j].operations.append(''.join(dp[i][j-1].operations) + " ins " + str(B[j-1]) + " ") 
            elif result_delete <= result_sub and result_delete <= result_insert:
                dp[i][j].value = result_delete
                dp[i][j].operations.append(''.join(dp[i-1][j].operations) + " del " + str(A[i-1]) + " ") 
            
    
    return dp[len(A)][len(B)]

In [104]:

# Examples LCS
A = "wednesday"
B = "nest"

a = len(A)
b = len(B)

# Create dp matrix 
dp = [[LEV_Entry() for j in range(b+1)] for i in range(a+1)]

result = LEV_BU(A, B)

print(result)
print(dp)


6:[' del w  del e  del d  keep n  keep e  keep s  del d  del a  sub y for t ']
[[-1:[], 1:[' ins n '], 2:[' ins n  ins e '], 3:[' ins n  ins e  ins s '], 4:[' ins n  ins e  ins s  ins t ']], [1:[' del w '], 0:[' sub w for n '], 1:[' sub w for n  ins e '], 2:[' sub w for n  ins e  ins s '], 3:[' sub w for n  ins e  ins s  ins t ']], [2:[' del w  del e '], 1:[' sub w for n  del e '], 0:[' sub w for n  keep e '], 1:[' sub w for n  keep e  ins s '], 2:[' sub w for n  keep e  ins s  ins t ']], [3:[' del w  del e  del d '], 2:[' sub w for n  del e  del d '], 1:[' sub w for n  keep e  del d '], 1:[' sub w for n  keep e  sub d for s '], 2:[' sub w for n  keep e  ins s  sub d for t ']], [4:[' del w  del e  del d  del n '], 3:[' del w  del e  del d  keep n '], 2:[' sub w for n  keep e  del d  del n '], 2:[' sub w for n  keep e  del d  sub n for s '], 2:[' sub w for n  keep e  sub d for s  sub n for t ']], [5:[' del w  del e  del d  del n  del e '], 4:[' del w  del e  del d  keep n  del e '], 3:[

# Probability of Subsequence

Let $S$ be a string of length $n$, and $k$ a positive integer (subsequence size). We wish to count at all $n-k+1$ positions the subsequence can overlap $S$ and count the number of times such a subsequence appears. We then convert these counts to probabilities by normalizing over $n-k-1$. 

In [34]:
S = "aabbabbbabbbabbbabbbbaaa"
n = len(S)
k = 3    # Window size

d = dict() # store counts of substrings in dictionary. 

# Count subsequences
for i in range(n-k+1):
    subseq = S[i:i+k]
    
    if subseq in d:
        d[subseq] = d.get(subseq) + 1
    else:
        d[subseq] = 1

# normalize    
for e in d.items():
    d[e[0]] = e[1] / (n-k+1)
    
# Print sorted by decreasing order of probability. 
print(sorted(d.items(), key=lambda count: count[1], reverse=True))
    

[('abb', 0.22727272727272727), ('bba', 0.22727272727272727), ('bbb', 0.22727272727272727), ('bab', 0.18181818181818182), ('aab', 0.045454545454545456), ('baa', 0.045454545454545456), ('aaa', 0.045454545454545456)]
